In [19]:
from bs4 import BeautifulSoup as bs
import re
import requests
from multiprocessing import Pool, Queue
import wget

In [20]:
data = []
with open("./brassicaceae.tsv") as f:
    while True:
        line = f.readline()
        try: 
            data.append([line.split("\t")[0].strip(), line.split("\t")[1].strip()])
        except IndexError:
            data.append([line.split("\t")[0].strip(), "-"])
        if not line: break

In [21]:
cycle = r"(.*erennial.*)!(.*nnual.*)" 
def crawl_once(element):
    try: 
        req = requests.get(f"http://worldfloraonline.org/taxon/{element[0]}", timeout=30)
    except requests.exceptions.ConnectTimeout:
        return None
    soup = bs(req.text, "html.parser")
    general = soup.select("#general")
    morphology = soup.select("#morphology")
    if (bool(re.compile(cycle).search(str(general))) or bool(re.compile(cycle).search(str(morphology)))): 
        return element[1]

In [22]:
pool = Pool(processes=80)
naive = pool.map(crawl_once, data)
target = []
for e in naive:
    if e != None: target.append(e)

In [23]:
with open("PerennialBrassicaceae.txt", "w") as f:
    for w in target:
        f.write(f"{w}\n")

In [25]:
target[0]

IndexError: list index out of range